# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv file cities.csv created in WeatherPy exercise
csvpath = os.path.join("..", "WeatherPy","output_data","cities.csv")
# Create new dataframe vacation_df
vacation_df = pd.read_csv(csvpath)
# Drop the Unamed: 0 column from the dataframe
vacation_df = vacation_df.drop("Unnamed: 0", axis="columns")
vacation_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hilo,19.7297,-155.0900,80.60,61,75,13.80,US,1619218925
1,Mar del Plata,-38.0023,-57.5575,69.80,78,0,6.91,AR,1619218926
2,Guerrero Negro,27.9769,-114.0611,64.04,62,10,22.10,MX,1619218926
3,Ushuaia,-54.8000,-68.3000,42.80,87,75,4.61,AR,1619218809
4,Berlevåg,70.8578,29.0864,33.03,84,81,13.80,NO,1619218815


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Store lat/Lng coordinates from dataframe in locations
locations = vacation_df[["Lat", "Lng"]]
# Convert humidity values to float
humid = vacation_df["Humidity"].astype(float)
# Plot the heatmap
figure_layout = {"width": "200px", "height": "150px", "border":"1px solid black"}

fig = gmaps.figure()
# Configure heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humid, dissipating=False, max_intensity = 30, point_radius=3)
                               
# Add heatmap to figure
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [3]:
# Set conditions for list of cities with ideal weather
ideal_temp = vacation_df.loc[(vacation_df["Max Temp"] < 70)&(vacation_df["Max Temp"] > 50),:]
ideal_humid = ideal_temp.loc[(ideal_temp["Humidity"] > 40) & (ideal_temp["Humidity"]<50),:]
ideal_clouds = ideal_humid.loc[ideal_humid["Cloudiness"] < 45,:]
ideal_clouds.dropna()
ideal_clouds

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,La Rochelle,46.1667,-1.1500,59.00,47,0,10.36,FR,1619219000
76,Shakawe,-18.3667,21.8500,61.86,43,1,6.22,BW,1619219005
116,Port Hedland,-20.3167,118.5667,64.40,48,0,5.75,AU,1619219048
173,Khani,41.9563,42.9566,57.20,47,0,9.22,GE,1619219149
245,Batemans Bay,-35.7167,150.1833,61.00,48,14,3.69,AU,1619219227
307,Wana,32.2989,69.5725,58.73,49,0,2.17,PK,1619219363
340,Gojra,31.1487,72.6866,68.09,49,0,5.99,PK,1619219371
428,Plouzané,48.3833,-4.6167,60.80,44,0,11.50,FR,1619219513
443,Emerald,-23.5333,148.1667,68.00,42,8,12.66,AU,1619219517
468,Mt. Pleasant,32.7941,-79.8626,64.99,48,1,10.49,US,1619219293


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [4]:
# Saving to new dataframe named hotels_df
hotel_df = ideal_clouds.copy()
hotel_df = hotel_df.reset_index(drop=True)
# Adding new blank column to hold hotel names
hotel_df["Hotel Name"] = ""

# Set parameters
hotel_radius = 5000
search_type = "lodging"
params = {"type": search_type, "radius": hotel_radius, "key": g_key}

# Start loop of rows in hotel_df to get nearest hotels based on lat/Long
for index, row in hotel_df.iterrows():
    # Establish base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Set the latitude and longitude for parameters
    lat_coord = row["Lat"]
    lon_coord = row["Lng"]
    params["location"] = f"{lat_coord},{lon_coord}"
    # Requests hotel data from google places 
    response = requests.get(base_url, params=params).json()
    hotel_results = response["results"]
    # Populate hotel name column with hotels
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_results[0]["name"]
    
    except:
        print("Missing hotel data....skipping")
        pass
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,La Rochelle,46.1667,-1.1500,59.00,47,0,10.36,FR,1619219000,Hotel Henri IV
1,Shakawe,-18.3667,21.8500,61.86,43,1,6.22,BW,1619219005,Ditlhapi Guest House
2,Port Hedland,-20.3167,118.5667,64.40,48,0,5.75,AU,1619219048,The Esplanade Hotel
3,Khani,41.9563,42.9566,57.20,47,0,9.22,GE,1619219149,კურორტი ზეკარი/Resort Zekari
4,Batemans Bay,-35.7167,150.1833,61.00,48,14,3.69,AU,1619219227,Batemans Bay Marina Resort
5,Wana,32.2989,69.5725,58.73,49,0,2.17,PK,1619219363,Rest House at Wana Camp
6,Gojra,31.1487,72.6866,68.09,49,0,5.99,PK,1619219371,Zee Go Hotel
7,Plouzané,48.3833,-4.6167,60.80,44,0,11.50,FR,1619219513,Carrefour Voyages Plouzané
8,Emerald,-23.5333,148.1667,68.00,42,8,12.66,AU,1619219517,The Irish Village
9,Mt. Pleasant,32.7941,-79.8626,64.99,48,1,10.49,US,1619219293,Best Western Patriots Point


In [5]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
gmaps.configure(api_key=g_key)
# Plot the heatmap
fig = gmaps.figure()
# Configure heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = hotel_humid, dissipating=False, max_intensity = 30, point_radius=3)

# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))